In [139]:
import pandas as pd
import numpy as np
mh_df = pd.read_csv('MH_cleaned.csv')
gs_df = pd.read_csv('greenspace_cleaned.csv')

In [140]:
mh_df.rename(columns={'StateAbbr': 'State', 'PlaceName': 'Cities in Urban Center'}, inplace=True)
mh_df.head()

,State,Cities in Urban Center,PlaceFIPS,Population2010,MHLTH_AdjPrev,MHLTH_Adj95CI,Geolocation
0,AL,Birmingham,107000,212237,15.6,"(15.4, 15.8)","[33.5275663773, -86.7988174678]"
1,AL,Hoover,135896,81619,10.4,"(10.1, 10.7)","[33.3767602729, -86.8051937568]"
2,AL,Huntsville,137000,180105,13.4,"(13.2, 13.7)","[34.6989692671, -86.6387042882]"
3,AL,Mobile,150000,195111,15.0,"(14.9, 15.2)","[30.6776248648, -88.1184482714]"
4,AL,Montgomery,151000,205764,14.8,"(14.6, 15.1)","[32.3472645333, -86.2677059552]"


In [141]:
gs_df.drop(columns=['Unnamed: 0', 'Country'], inplace=True)
mh_df.drop(columns=['Geolocation', 'MHLTH_Adj95CI', 'PlaceFIPS'], inplace=True)

In [142]:
t1 = gs_df[gs_df['Cities in Urban Center'] == 'San Francisco']
t2 = mh_df[mh_df['Cities in Urban Center'] == 'San Francisco']
t2

,State,Cities in Urban Center,Population2010,MHLTH_AdjPrev
112,CA,San Francisco,805235,9.8


In [143]:
df = pd.merge(mh_df, gs_df, on=['Cities in Urban Center', 'State'], how='inner')
df.head()

,State,Cities in Urban Center,Population2010,MHLTH_AdjPrev,UC Grouping,Latitude,Longitude,Urban Center,E_GR_AV14,E_GR_AT14,...,SDG_OS15MX,P15,B15,BUCAP15,INCM_CMI,DEV_CMI,GDP15_SM,E_BM_NM_LST,E_WR_T_14,Cities in Urban Center_copy
0,AL,Birmingham,212237,15.6,485,33.509025,-86.823651,Birmingham,0.494568,219.996230,...,74.85,196387.76700,152.894608,778.534274,HIC,MDR,6.184143e+09,Temperate Broadleaf and Mixed Forests,17.497644,Birmingham;
1,AL,Huntsville,180105,13.4,501,34.726065,-86.609995,Huntsville,0.521522,88.700999,...,66.37,86467.06209,59.674004,690.135667,HIC,MDR,2.498489e+09,Temperate Broadleaf and Mixed Forests,16.321889,Huntsville
2,AL,Mobile,195111,15.0,422,30.692377,-88.093685,Mobile,0.467515,122.669298,...,63.32,118578.67890,71.298004,601.271703,HIC,MDR,4.072112e+09,Temperate Coniferous Forests,20.312027,Mobile
3,AL,Montgomery,205764,14.8,481,32.360134,-86.249187,Montgomery,0.512807,153.986442,...,75.47,143898.04020,92.235245,640.976379,HIC,MDR,4.288399e+09,Temperate Broadleaf and Mixed Forests,18.804660,Montgomery
4,AL,Tuscaloosa,90468,15.5,470,33.199564,-87.553567,Tuscaloosa,0.439016,44.506974,...,41.24,57182.06445,26.439175,462.368313,HIC,MDR,1.541527e+09,Temperate Broadleaf and Mixed Forests,17.963472,Tuscaloosa


In [144]:
df.shape

(340, 21)

In [145]:
df_alpha = df[['UC Grouping', 'State', 'INCM_CMI', 'DEV_CMI', 'E_BM_NM_LST', 'Cities in Urban Center_copy']].copy()
df.drop(columns=['State', 'Cities in Urban Center', 'Urban Center', 'INCM_CMI', 'DEV_CMI', 'E_BM_NM_LST', 'Cities in Urban Center_copy'], inplace=True)

In [146]:
cols = list(df.columns)
cols.remove('Population2010')
print(cols)

['MHLTH_AdjPrev', 'UC Grouping', 'Latitude', 'Longitude', 'E_GR_AV14', 'E_GR_AT14', 'SDG_A2G14', 'SDG_OS15MX', 'P15', 'B15', 'BUCAP15', 'GDP15_SM', 'E_WR_T_14']


In [147]:
aggregations = {'Population2010': 'sum'}
for i in cols:
    aggregations[i] = 'mean'
grouped = df.groupby(by='UC Grouping').agg(aggregations)

In [148]:
grouped.reset_index(drop=True)
grouped['UC Grouping'] = grouped['UC Grouping'].astype(int)

In [149]:
print(df_alpha.shape)
df_alpha.drop_duplicates(keep='first', inplace=True)
print(df_alpha.shape)

(340, 6)
(235, 6)


In [150]:
df_alpha.head()

,UC Grouping,State,INCM_CMI,DEV_CMI,E_BM_NM_LST,Cities in Urban Center_copy
0,485,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Birmingham;
1,501,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Huntsville
2,422,AL,HIC,MDR,Temperate Coniferous Forests,Mobile
3,481,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Montgomery
4,470,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Tuscaloosa


In [151]:
dfagg = pd.merge(df, df_alpha, on=['UC Grouping'], how='inner')
dfagg.head()

,Population2010,MHLTH_AdjPrev,UC Grouping,Latitude,Longitude,E_GR_AV14,E_GR_AT14,SDG_A2G14,SDG_OS15MX,P15,B15,BUCAP15,GDP15_SM,E_WR_T_14,State,INCM_CMI,DEV_CMI,E_BM_NM_LST,Cities in Urban Center_copy
0,212237,15.6,485,33.509025,-86.823651,0.494568,219.996230,0.773812,74.85,196387.76700,152.894608,778.534274,6.184143e+09,17.497644,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Birmingham;
1,180105,13.4,501,34.726065,-86.609995,0.521522,88.700999,0.802599,66.37,86467.06209,59.674004,690.135667,2.498489e+09,16.321889,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Huntsville
2,195111,15.0,422,30.692377,-88.093685,0.467515,122.669298,0.822213,63.32,118578.67890,71.298004,601.271703,4.072112e+09,20.312027,AL,HIC,MDR,Temperate Coniferous Forests,Mobile
3,205764,14.8,481,32.360134,-86.249187,0.512807,153.986442,0.747151,75.47,143898.04020,92.235245,640.976379,4.288399e+09,18.804660,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Montgomery
4,90468,15.5,470,33.199564,-87.553567,0.439016,44.506974,0.411681,41.24,57182.06445,26.439175,462.368313,1.541527e+09,17.963472,AL,HIC,MDR,Temperate Broadleaf and Mixed Forests,Tuscaloosa


In [152]:
dfagg.drop(columns=['UC Grouping'], inplace=True)
dfagg.to_csv('merged_greenspace_mh.csv')